In [1]:
# IMPORTS
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

In [2]:
# VARIABLES
landings = ["daveys-locker","dana-wharf-sportfishing"]
landingID = {"daveys-locker":"16","dana-wharf-sportfishing":"17"}
url = "https://www.sportfishingreport.com/landings/"
# URL FORMATS = https://www.sportfishingreport.com/landings/daveys-locker.php?landing_id=16&year=2020&month=6&day=5
#         or   https://www.sportfishingreport.com/landings/dana-wharf-sportfishing.php?landing_id=17&year=2019&month=1&day=1
start_date = date(2019,1,1)
end_date = date(2020,10,28)
dateRange = pd.date_range(start_date, end_date)

In [3]:
df = pd.DataFrame(columns=["Landing","Date","Boat","Trip Type","Anglers","Fish Count"])

In [4]:
# loop over landings
for landing in landings:
    # loop over dates
    for date in dateRange:
        # get specific date url
        result = requests.get(url + landing + ".php?" + "&year=" + str(date.year) +
                             "&month=" + str(date.month) + "&day=" + str(date.day))
        # check for valid html return status
        if(result.status_code == 200):
            # get src
            src = result.content
            soup = BeautifulSoup(src,'lxml')
            # read in "td" tags
            links = soup.find_all("td")
            i=1
            while(True):
                if("Dock Totals" in links[i*4+1].text):
                    break
                else:
                    df = df.append({"Landing":landing,"Date":date.strftime("%Y-%m-%d"),
                                    "Boat":links[i*4+1].text,"Trip Type":links[i*4+2].text,
                                    "Anglers":links[i*4+3].text,"Fish Count":links[i*4+4].text}, ignore_index=True)
                    i += 1
            if date.day == 1:
                print(date.strftime("%Y-%#m-%#d"))

2019-1-1
2019-2-1
2019-3-1
2019-4-1
2019-5-1
2019-6-1
2019-7-1
2019-8-1
2019-9-1
2019-10-1
2019-11-1
2019-12-1
2020-1-1
2020-2-1
2020-3-1
2020-4-1
2020-5-1
2020-6-1
2020-7-1
2020-8-1
2020-9-1
2020-10-1
2019-1-1
2019-2-1
2019-3-1
2019-4-1
2019-5-1
2019-6-1
2019-7-1
2019-8-1
2019-9-1
2019-10-1
2019-11-1
2019-12-1
2020-1-1
2020-2-1
2020-3-1
2020-4-1
2020-5-1
2020-6-1
2020-7-1
2020-8-1
2020-9-1
2020-10-1


In [5]:
df.head()

,Landing,Date,Boat,Trip Type,Anglers,Fish Count
0,daveys-locker,2019-01-11,Western Pride,1/2 Day PM,35,"38 Sand Bass, 16 Sculpin, 10 Blacksmith"
1,daveys-locker,2019-01-12,Western Pride,1/2 Day Twilight,66,"138 Sculpin, 28 Sand Bass"
2,daveys-locker,2019-01-19,Western Pride,1/2 Day AM,76,"150 Sculpin Released, 102 Sculpin, 40 Sanddab"
3,daveys-locker,2019-01-19,Western Pride,1/2 Day Twilight,67,"156 Sculpin, 29 Sand Bass, 5 Black Croaker, 3 ..."
4,daveys-locker,2019-01-20,Western Pride,1/2 Day AM,62,161 Sculpin


In [6]:
df.to_csv("Data/fishCounts.csv")